# 09 - Calibration Platt Isotonic

Goals:
- Load the data `features_match_long_elo_22_23`
- Choose X (features) and y (target = result H/D/A)
- Split train/test by time order
- Train my 3 model :
    - Logistic regression
    - Random Forest
    - XGBoost
- Apply :
    - No calibration (raw)
    - Platt scaling (sigmoid)
    - Isotonic regression
- For each, calculate :
    - Accuracy
    - Log Loss
    - Brier Score
- Compare all results in a single summary table

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss, brier_score_loss
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

import matplotlib.pyplot as plt
import xgboost as xgb

pd.set_option("display.max_columns", 50)

# Load long-format features with Elo (two rows per match, one per team)
df = pd.read_csv(FEATURES_ELO_PATH)
print("Long + Elo features shape:", df.shape)
df.head()


In [ ]:
target_col = "result"

feature_cols = [
    "is_home",
    "rolling_xg_for_5",
    "rolling_xg_against_5",
    "rolling_xg_diff_5",
    "rolling_points_5",
    "strength_points_5",
    "elo_team_before",
    "elo_diff_for_team",
]

# Check NaNs before dropping
print("NaN per column BEFORE drop:", df.shape)
print(df[feature_cols + [target_col]].isna().sum())

# Drop rows with missing values in features or target
df_clean = df.dropna(subset=feature_cols + [target_col]).reset_index(drop=True)

print("\nShape AFTER drop:", df_clean.shape)
print("NaN per column AFTER drop:")
print(df_clean[feature_cols + [target_col]].isna().sum())

# Ensure date is datetime and sort chronologically
df_clean["date"] = pd.to_datetime(df_clean["date"])
df_clean = df_clean.sort_values("date").reset_index(drop=True)

X = df_clean[feature_cols].copy()
y = df_clean[target_col].copy()

print("\nX shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
train_ratio = 0.8
n = len(df_clean)
train_size = int(train_ratio * n)

X_train = X.iloc[:train_size]
X_test  = X.iloc[train_size:]
y_train = y.iloc[:train_size]
y_test  = y.iloc[train_size:]

print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes :", X_test.shape, y_test.shape)

# Encode target labels (H/D/A) into integers
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc  = le.transform(y_test)

print("\nClasses:", le.classes_)     # expected ['A', 'D', 'H']
print("First 10 encoded y_train:", y_train_enc[:10])

In [ ]:
# Scale features for logistic regression only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("Scaled shapes:", X_train_scaled.shape, X_test_scaled.shape)

In [ ]:
# Compute accuracy, log loss and Brier scores for given probabilities.
def evaluate_probas(y_test_enc: np.ndarray, y_proba: np.ndarray, le: LabelEncoder, label: str = "") -> dict:
    # Predicted classes from probabilities
    y_pred_enc = np.argmax(y_proba, axis=1)

    accuracy = accuracy_score(y_test_enc, y_pred_enc)
    ll = log_loss(y_test_enc, y_proba)

    # Brier scores per class
    brier_scores = []
    for class_idx, class_label in enumerate(le.classes_):
        y_true_binary = (y_test_enc == class_idx).astype(int)
        y_prob_class = y_proba[:, class_idx]
        brier = brier_score_loss(y_true_binary, y_prob_class)
        brier_scores.append(brier)

    mean_brier = float(np.mean(brier_scores))

    print(f"\n=== {label} ===")
    print(f"Accuracy : {accuracy:.3f}")
    print(f"Log loss : {ll:.4f}")
    print(f"Mean Brier score: {mean_brier:.4f}")

    return {
        "accuracy": float(accuracy),
        "log_loss": float(ll),
        "brier_scores": brier_scores,
        "mean_brier": mean_brier,
    }

In [ ]:
# Base logistic regression model (multinomial)
logit = LogisticRegression(
    multi_class="multinomial",
    solver="lbfgs",
    max_iter=500,
    random_state=42,
)

logit.fit(X_train_scaled, y_train_enc)
y_proba_logit_raw = logit.predict_proba(X_test_scaled)

metrics_logit_raw = evaluate_probas(
    y_test_enc,
    y_proba_logit_raw,
    le,
    label="Logistic (raw, uncalibrated)",
)

In [ ]:
# Logistic regression with Platt scaling (sigmoid)
logit_platt = CalibratedClassifierCV(
    estimator=LogisticRegression(
        multi_class="multinomial",
        solver="lbfgs",
        max_iter=500,
        random_state=42,
    ),
    method="sigmoid",   # Platt scaling
    cv=3,               # 3-fold CV on training set
)

logit_platt.fit(X_train_scaled, y_train_enc)
y_proba_logit_platt = logit_platt.predict_proba(X_test_scaled)

metrics_logit_platt = evaluate_probas(
    y_test_enc,
    y_proba_logit_platt,
    le,
    label="Logistic + Platt (sigmoid)",
)

In [ ]:
# Logistic regression with isotonic calibration
logit_iso = CalibratedClassifierCV(
    estimator=LogisticRegression(
        multi_class="multinomial",
        solver="lbfgs",
        max_iter=500,
        random_state=42,
    ),
    method="isotonic",
    cv=3,
)

logit_iso.fit(X_train_scaled, y_train_enc)
y_proba_logit_iso = logit_iso.predict_proba(X_test_scaled)

metrics_logit_iso = evaluate_probas(
    y_test_enc,
    y_proba_logit_iso,
    le,
    label="Logistic + Isotonic",
)

In [ ]:
# Base Random Forest model (uncalibrated)
rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1,
)

rf.fit(X_train, y_train_enc)
y_proba_rf_raw = rf.predict_proba(X_test)

metrics_rf_raw = evaluate_probas(
    y_test_enc,
    y_proba_rf_raw,
    le,
    label="Random Forest (raw, uncalibrated)",
)

In [ ]:
# Random Forest with Platt scaling
rf_platt = CalibratedClassifierCV(
    estimator=RandomForestClassifier(
        n_estimators=500,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1,
    ),
    method="sigmoid",
    cv=3,
)

rf_platt.fit(X_train, y_train_enc)
y_proba_rf_platt = rf_platt.predict_proba(X_test)

metrics_rf_platt = evaluate_probas(
    y_test_enc,
    y_proba_rf_platt,
    le,
    label="Random Forest + Platt (sigmoid)",
)

In [ ]:
# Random Forest with isotonic calibration
rf_iso = CalibratedClassifierCV(
    estimator=RandomForestClassifier(
        n_estimators=500,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1,
    ),
    method="isotonic",
    cv=3,
)

rf_iso.fit(X_train, y_train_enc)
y_proba_rf_iso = rf_iso.predict_proba(X_test)

metrics_rf_iso = evaluate_probas(
    y_test_enc,
    y_proba_rf_iso,
    le,
    label="Random Forest + Isotonic",
)


In [ ]:
# Base XGBoost model (uncalibrated)
xgb_model = xgb.XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    n_estimators=400,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    eval_metric="mlogloss",
)

xgb_model.fit(X_train, y_train_enc)
y_proba_xgb_raw = xgb_model.predict_proba(X_test)

metrics_xgb_raw = evaluate_probas(
    y_test_enc,
    y_proba_xgb_raw,
    le,
    label="XGBoost (raw, uncalibrated)",
)

In [ ]:
# XGBoost with Platt scaling
xgb_platt = CalibratedClassifierCV(
    estimator=xgb.XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        n_estimators=400,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        eval_metric="mlogloss",
    ),
    method="sigmoid",
    cv=3,
)

xgb_platt.fit(X_train, y_train_enc)
y_proba_xgb_platt = xgb_platt.predict_proba(X_test)

metrics_xgb_platt = evaluate_probas(
    y_test_enc,
    y_proba_xgb_platt,
    le,
    label="XGBoost + Platt (sigmoid)",
)

In [ ]:
# XGBoost with isotonic calibration
xgb_iso = CalibratedClassifierCV(
    estimator=xgb.XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        n_estimators=400,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        eval_metric="mlogloss",
    ),
    method="isotonic",
    cv=3,
)

xgb_iso.fit(X_train, y_train_enc)
y_proba_xgb_iso = xgb_iso.predict_proba(X_test)

metrics_xgb_iso = evaluate_probas(
    y_test_enc,
    y_proba_xgb_iso,
    le,
    label="XGBoost + Isotonic",
)

In [ ]:
summary = []

def add_row(model_name, calib_type, metrics):
    summary.append({
        "model": model_name,
        "calibration": calib_type,
        "accuracy": metrics["accuracy"],
        "log_loss": metrics["log_loss"],
        "mean_brier": metrics["mean_brier"],
    })

add_row("Logistic", "raw",      metrics_logit_raw)
add_row("Logistic", "Platt",    metrics_logit_platt)
add_row("Logistic", "Isotonic", metrics_logit_iso)

add_row("RandomForest", "raw",      metrics_rf_raw)
add_row("RandomForest", "Platt",    metrics_rf_platt)
add_row("RandomForest", "Isotonic", metrics_rf_iso)

add_row("XGBoost", "raw",      metrics_xgb_raw)
add_row("XGBoost", "Platt",    metrics_xgb_platt)
add_row("XGBoost", "Isotonic", metrics_xgb_iso)

df_summary = pd.DataFrame(summary)
df_summary.sort_values(["model", "log_loss"])